In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import ToTensor

print(f'Pytorch version:  {torch.__version__}')

ModuleNotFoundError: No module named 'torch'